Setup:

In [ ]:
%pip install -Uqq fastai fastbook graphviz

import fastbook
from fastai.vision.all import *
from fastbook import *

fastbook.setup_book()

Import MNIST sample that contains samples of all numbers from 0 to 9:

In [ ]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
(path/'testing').ls()

Create dictionary `digits` of all digits in MNIST sample:

In [ ]:
import os

dir = os.listdir(path/'training')
dir_training = os.listdir(path/'testing')

digits = {}
for digit in dir:
    digits[digit] = (path/'training'/digit).ls().sorted()
    
digits_training = {}
for digit in dir_training:
    digits_training[digit] = (path/'testing'/digit).ls().sorted()

Create `mnist_loss` function:

In [ ]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

Create function to initiate random parameters:

In [ ]:
def init_params(size, std=1.0): 
    return (torch.randn(size)*std).requires_grad_()

Create function `weight_bias_pairs` to create `number_pairs` number of weight-bias pairs as tuples:

In [ ]:
def weight_bias_pairs(number_pairs):
    pair_list = []
    for i in range(number_pairs, 0, -1):
        w = init_params((pow(2, i), pow(2, i-1)))
        b = init_params((pow(2, i-1)))
        pair_list.append((w, b))
    return pair_list

Create function `neural_net` for creating a neural_network using the `weight_bias_pairs` function:

In [ ]:
def neural_net(number_pairs):
    def neural_net_inner(xb):
        wb_pair_list = weight_bias_pairs(number_pairs)
        es = xb@wb_pair_list[0][0] + wb_pair_list[0][1]
        for wb_pair in wb_pair_list[:1]:
            res = res.max(tensor(0.0))
            res = res@wb_pair[0] + wb_pair[1]
        return res
    return neural_net_inner

Create function `batch_accuracy` to test validation accuracy of batch:

In [ ]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

Create data in `digits` elements into tuples containing stacked tensors and the original images:

In [ ]:
for digit, data in digits.items():
    digit_tensor = [tensor(Image.open(o)) for o in data]
    stacked_digit = torch.stack(digit_tensor).float()/255
    digits[digit] = (stacked_digit, data)

Create training set `train_x` containing each digit in the set:

In [ ]:
train_x = torch.cat([i[0] for i in digits.values()]).view(-1, 28*28)

Create a training tensor cainting the amounts of each image and show both:

In [ ]:
train_y = tensor([0]*len(digits['0'][1]) + [1]*len(digits['1'][1]) + [2]*len(digits['2'][1]) + [3]*len(digits['3'][1]) + [4]*len(digits['4'][1]) + [5]*len(digits['5'][1]) + [6]*len(digits['6'][1]) + [7]*len(digits['7'][1]) + [8]*len(digits['8'][1]) + [9]*len(digits['9'][1])).unsqueeze(1)
train_x.shape,train_y.shape

Create dataset of `train_x` and `train_y`:

In [ ]:
dset = list(zip(train_x,train_y))

Create a `DataLoader` from `Dataset`:

In [ ]:
dl = DataLoader(dset, batch_size=1024)
xb,yb = first(dl)
xb.shape,yb.shape

Repeat the process for the validation data:

In [ ]:
for digit, data in digits_training.items():
    digit_tensor = [tensor(Image.open(o)) for o in data]
    stacked_digit = torch.stack(digit_tensor).float()/255
    digits_training[digit] = (stacked_digit, data)
    
valid_x = torch.cat([i[0] for i in digits_training.values()]).view(-1, 28*28)
valid_y = tensor([0]*len(digits_training['0'][1]) + [1]*len(digits_training['1'][1]) + [2]*len(digits_training['2'][1]) + [3]*len(digits_training['3'][1]) + [4]*len(digits_training['4'][1]) + [5]*len(digits_training['5'][1]) + [6]*len(digits_training['6'][1]) + [7]*len(digits_training['7'][1]) + [8]*len(digits_training['8'][1]) + [9]*len(digits_training['9'][1])).unsqueeze(1)

valid_x,valid_y

In [ ]:
valid_dset = list(zip(valid_x, valid_y))
valid_dl = DataLoader(valid_dset, batch_size=1024)

Create dataloader of both training and validation data:

In [ ]:
dls = DataLoaders(dl, valid_dl)

Create `Learner` function to be trained:

In [ ]:
learn = Learner(dls, neural_net(10), opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)

Train the model:

In [ ]:
learn.fit(40, lr=0.1)